In [2]:
from functions.read import *
loans, recharges = load_dfs()

# Falta fazer

* Criar os cenários para escolher qual melhor modelo
* Relátorio final
* Criar os mocks de teste do feature eng
* Se der tempo fazer um CLI básico

# Exploratory data analysis

É interessante observar que os emprestimos não pagos não varia muito da mediana populacional, 5, entretanto utilizar essa informação pode gerar bias no modelo preditivo.


LEMBRETES:
    Criar intervalos de loans permitidos para cada usuário
    Fazes dois mocks para as funções de feature eng

Notas:
    Distribuição acumulado inad. 
        target_sum
        0    7044
        1    2463
        2       5
        4       1

#t_loans


In [2]:
#print('General Loans stats \n',t_loans['amount'].describe(),'\n',f"Median : {t_loans['amount'].median()}")
#print('Paid stats \n',t_loans.query('target == 0').groupby('uuid').sum('amount').sort_values('amount',ascending=False)['amount'].describe())
#print('Not Paid stats \n',t_loans.query('target == 1').groupby('uuid').sum('amount').sort_values('amount',ascending=False)['amount'].describe())

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from functions.feature_default import ForecastDefault

s = ForecastDefault(
    loans_hist=loans, 
    recharges_hist=recharges, estimators_list=[
                                                LogisticRegression,
                                                XGBClassifier,
                                                RandomForestClassifier,
                                                ],
    eval_metric='f1',
    inicial_date='2019-01-01',days_to_default=60,limit_date='2019-12-05')

last_date 2019-10-06 00:00:00
limit_date 2019-12-04 23:54:57.280521
train_loans:  (2522, 5)
test_loans:  (16439, 5)
Dist train_y : 
 0    119
1    119
Name: target, dtype: int64
train_X:  (238, 4)
train_y:  (238,)
test_X:  (238, 4)
test_y:  (238,)
<class 'sklearn.linear_model._logistic.LogisticRegression'> : Training sample metrics | [0.75, 0.667, 0.846, 0.784, 0.727] | mean: 0.755
<class 'xgboost.sklearn.XGBClassifier'> : Training sample metrics | [0.809, 0.8, 0.851, 0.88, 0.857] | mean: 0.839
<class 'sklearn.ensemble._forest.RandomForestClassifier'> : Training sample metrics | [0.844, 0.75, 0.889, 0.88, 0.826] | mean: 0.838
Best model :XGBClassifier
Test sampling f1 : 0.033
Classification report 
               precision    recall  f1-score   support

           0       0.99      0.50      0.67       235
           1       0.02      0.67      0.03         3

    accuracy                           0.50       238
   macro avg       0.50      0.58      0.35       238
weighted avg       